In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl

import math
from scipy.stats import zscore
from numpy import nanmean, nanstd


from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier

from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline

import lightgbm as lgb
import catboost as cb
import xgboost as xgb

import itertools

import shap
import pickle

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import torch

import optuna

In [2]:
SHAP_LGB = False
SHAP_CB = False
SHAP_XGB = True

In [3]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

In [4]:
df_train = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_train.head()

Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,tbp_lv_Cext,tbp_lv_H,tbp_lv_Hext,tbp_lv_L,tbp_lv_Lext,tbp_lv_areaMM2,tbp_lv_area_perim_ratio,tbp_lv_color_std_mean,tbp_lv_deltaA,tbp_lv_deltaB,tbp_lv_deltaL,tbp_lv_deltaLB,tbp_lv_deltaLBnorm,tbp_lv_eccentricity,tbp_lv_location,tbp_lv_location_simple,tbp_lv_minorAxisMM,tbp_lv_nevi_confidence,tbp_lv_norm_border,tbp_lv_norm_color,tbp_lv_perimeterMM,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,16.261975,26.922447,23.954773,33.684638,28.953117,53.058545,55.828924,54.367448,62.025701,3.152561,27.476170,0.00000,3.982447,2.967674,-7.658253,8.360566,5.784302,0.901302,Right Leg - Upper,Right Leg,1.543016,2.628592e-03,7.091360,0.000000,9.307003,0.00000,2.036195,2.637780,0.590476,85,-182.703552,613.493652,-42.427948,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,25.364740,26.331000,24.549290,41.219030,35.299260,39.702910,44.064040,48.861520,55.362360,0.919497,12.235290,0.00000,6.347830,1.781713,-6.500838,6.839008,4.987244,0.639885,Head & Neck,Head & Neck,0.821918,1.334303e-07,2.116402,0.000000,3.354148,0.00000,0.853227,3.912844,0.285714,55,-0.078308,1575.687000,57.174500,Memorial Sloan Kettering Cancer Center,CC-BY,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,17.128170,37.970460,33.485410,44.174920,37.611800,59.265850,62.909730,53.961180,61.670520,3.265153,24.184620,0.00000,5.447655,4.485044,-7.709336,9.092376,6.290359,0.932147,Torso Back Top Third,Torso Back,1.194905,2.959177e-04,4.798335,0.000000,8.886309,0.00000,1.743651,1.950777,0.361905,105,123.649700,1472.010000,232.908900,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,12.164757,21.448144,21.121356,25.746200,24.374023,56.414429,60.060388,18.649518,23.314841,6.079940,14.889242,0.51452,2.077572,0.326788,-4.665323,4.783413,6.400196,0.654458,Torso Front Top Half,Torso Front,2.481328,2.198945e+01,1.975874,1.771705,9.514499,0.66469,1.258541,1.573733,0.209581,130,-141.024780,1442.185791,58.359802,ACEMID MIA,CC-0,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,20.057470,26.464900,25.710460,36.217980,32.608740,46.946070,52.041180,46.276310,54.855740,2.101708,19.902560,0.00000,4.668053,0.754434,-8.579431,9.148495,6.531302,0.946448,Torso Front Top Half,Torso Front,0.929916,1.378832e-03,3.658854,0.000000,6.467562,0.00000,2.085409,2.480509,0.313433,20,-72.315640,1488.720000,21.428960,Memorial Sloan Kettering Cancer Center,CC-BY,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [5]:
# !python /kaggle/input/script-5-fold-effnetv1b0/main.py
# !mv submission.csv submission_effnetv1b0_oof.csv

In [6]:
root = Path('/kaggle/input/isic-2024-challenge')

train_path = root / 'train-metadata.csv'
test_path = root / 'test-metadata.csv'
subm_path = root / 'sample_submission.csv'

id_col = 'isic_id'
target_col = 'target'
group_col = 'patient_id'

err = 1e-5
sampling_ratio = 0.01
seed = 42

do_ud = True


num_cols = [
    'age_approx',                        # Approximate age of patient at time of imaging.
    'clin_size_long_diam_mm',            # Maximum diameter of the lesion (mm).+
    'tbp_lv_A',                          # A inside  lesion.+
    'tbp_lv_Aext',                       # A outside lesion.+
    'tbp_lv_B',                          # B inside  lesion.+
    'tbp_lv_Bext',                       # B outside lesion.+ 
    'tbp_lv_C',                          # Chroma inside  lesion.+
    'tbp_lv_Cext',                       # Chroma outside lesion.+
    'tbp_lv_H',                          # Hue inside the lesion; calculated as the angle of A* and B* in LAB* color space. Typical values range from 25 (red) to 75 (brown).+
    'tbp_lv_Hext',                       # Hue outside lesion.+
    'tbp_lv_L',                          # L inside lesion.+
    'tbp_lv_Lext',                       # L outside lesion.+
    'tbp_lv_areaMM2',                    # Area of lesion (mm^2).+
    'tbp_lv_area_perim_ratio',           # Border jaggedness, the ratio between lesions perimeter and area. Circular lesions will have low values; irregular shaped lesions will have higher values. Values range 0-10.+
    'tbp_lv_color_std_mean',             # Color irregularity, calculated as the variance of colors within the lesion's boundary.
    'tbp_lv_deltaA',                     # Average A contrast (inside vs. outside lesion).+
    'tbp_lv_deltaB',                     # Average B contrast (inside vs. outside lesion).+
    'tbp_lv_deltaL',                     # Average L contrast (inside vs. outside lesion).+
    'tbp_lv_deltaLB',                    #
    'tbp_lv_deltaLBnorm',                # Contrast between the lesion and its immediate surrounding skin. Low contrast lesions tend to be faintly visible such as freckles; high contrast lesions tend to be those with darker pigment. Calculated as the average delta LB of the lesion relative to its immediate background in LAB* color space. Typical values range from 5.5 to 25.+
    'tbp_lv_eccentricity',               # Eccentricity.+
    'tbp_lv_minorAxisMM',                # Smallest lesion diameter (mm).+
    'tbp_lv_nevi_confidence',            # Nevus confidence score (0-100 scale) is a convolutional neural network classifier estimated probability that the lesion is a nevus. The neural network was trained on approximately 57,000 lesions that were classified and labeled by a dermatologist.+,++
    'tbp_lv_norm_border',                # Border irregularity (0-10 scale); the normalized average of border jaggedness and asymmetry.+
    'tbp_lv_norm_color',                 # Color variation (0-10 scale); the normalized average of color asymmetry and color irregularity.+
    'tbp_lv_perimeterMM',                # Perimeter of lesion (mm).+
    'tbp_lv_radial_color_std_max',       # Color asymmetry, a measure of asymmetry of the spatial distribution of color within the lesion. This score is calculated by looking at the average standard deviation in LAB* color space within concentric rings originating from the lesion center. Values range 0-10.+
    'tbp_lv_stdL',                       # Standard deviation of L inside  lesion.+
    'tbp_lv_stdLExt',                    # Standard deviation of L outside lesion.+
    'tbp_lv_symm_2axis',                 # Border asymmetry; a measure of asymmetry of the lesion's contour about an axis perpendicular to the lesion's most symmetric axis. Lesions with two axes of symmetry will therefore have low scores (more symmetric), while lesions with only one or zero axes of symmetry will have higher scores (less symmetric). This score is calculated by comparing opposite halves of the lesion contour over many degrees of rotation. The angle where the halves are most similar identifies the principal axis of symmetry, while the second axis of symmetry is perpendicular to the principal axis. Border asymmetry is reported as the asymmetry value about this second axis. Values range 0-10.+
    'tbp_lv_symm_2axis_angle',           # Lesion border asymmetry angle.+
    'tbp_lv_x',                          # X-coordinate of the lesion on 3D TBP.+
    'tbp_lv_y',                          # Y-coordinate of the lesion on 3D TBP.+
    'tbp_lv_z',                          # Z-coordinate of the lesion on 3D TBP.+
]

new_num_cols = [
    'lesion_size_ratio',             # tbp_lv_minorAxisMM      / clin_size_long_diam_mm
    'lesion_shape_index',            # tbp_lv_areaMM2          / tbp_lv_perimeterMM **2
    'hue_contrast',                  # tbp_lv_H                - tbp_lv_Hext              abs
    'luminance_contrast',            # tbp_lv_L                - tbp_lv_Lext              abs
    'lesion_color_difference',       # tbp_lv_deltaA **2       + tbp_lv_deltaB **2 + tbp_lv_deltaL **2  sqrt  
    'border_complexity',             # tbp_lv_norm_border      + tbp_lv_symm_2axis
    'color_uniformity',              # tbp_lv_color_std_mean   / tbp_lv_radial_color_std_max

    'position_distance_3d',          # tbp_lv_x **2 + tbp_lv_y **2 + tbp_lv_z **2  sqrt
    'perimeter_to_area_ratio',       # tbp_lv_perimeterMM      / tbp_lv_areaMM2
    'area_to_perimeter_ratio',       # tbp_lv_areaMM2          / tbp_lv_perimeterMM
    'lesion_visibility_score',       # tbp_lv_deltaLBnorm      + tbp_lv_norm_color
    'symmetry_border_consistency',   # tbp_lv_symm_2axis       * tbp_lv_norm_border
    'consistency_symmetry_border',   # tbp_lv_symm_2axis       * tbp_lv_norm_border / (tbp_lv_symm_2axis + tbp_lv_norm_border)

    'color_consistency',             # tbp_lv_stdL             / tbp_lv_Lext
    'consistency_color',             # tbp_lv_stdL*tbp_lv_Lext / tbp_lv_stdL + tbp_lv_Lext
    'size_age_interaction',          # clin_size_long_diam_mm  * age_approx
    'hue_color_std_interaction',     # tbp_lv_H                * tbp_lv_color_std_mean
    'lesion_severity_index',         # tbp_lv_norm_border      + tbp_lv_norm_color + tbp_lv_eccentricity / 3
    'shape_complexity_index',        # border_complexity       + lesion_shape_index
    'color_contrast_index',          # tbp_lv_deltaA + tbp_lv_deltaB + tbp_lv_deltaL + tbp_lv_deltaLBnorm

    'log_lesion_area',               # tbp_lv_areaMM2          + 1  np.log
    'normalized_lesion_size',        # clin_size_long_diam_mm  / age_approx
    'mean_hue_difference',           # tbp_lv_H                + tbp_lv_Hext    / 2
    'std_dev_contrast',              # tbp_lv_deltaA **2 + tbp_lv_deltaB **2 + tbp_lv_deltaL **2   / 3  np.sqrt
    'color_shape_composite_index',   # tbp_lv_color_std_mean   + bp_lv_area_perim_ratio + tbp_lv_symm_2axis   / 3
    'lesion_orientation_3d',         # tbp_lv_y                , tbp_lv_x  np.arctan2
    'overall_color_difference',      # tbp_lv_deltaA           + tbp_lv_deltaB + tbp_lv_deltaL   / 3

    'symmetry_perimeter_interaction',# tbp_lv_symm_2axis       * tbp_lv_perimeterMM
    'comprehensive_lesion_index',    # tbp_lv_area_perim_ratio + tbp_lv_eccentricity + bp_lv_norm_color + tbp_lv_symm_2axis   / 4
    'color_variance_ratio',          # tbp_lv_color_std_mean   / tbp_lv_stdLExt
    'border_color_interaction',      # tbp_lv_norm_border      * tbp_lv_norm_color
    'border_color_interaction_2',
    'size_color_contrast_ratio',     # clin_size_long_diam_mm  / tbp_lv_deltaLBnorm
    'age_normalized_nevi_confidence',# tbp_lv_nevi_confidence  / age_approx
    'age_normalized_nevi_confidence_2',
    'color_asymmetry_index',         # tbp_lv_symm_2axis       * tbp_lv_radial_color_std_max

    'volume_approximation_3d',       # tbp_lv_areaMM2          * sqrt(tbp_lv_x**2 + tbp_lv_y**2 + tbp_lv_z**2)
    'color_range',                   # abs(tbp_lv_L - tbp_lv_Lext) + abs(tbp_lv_A - tbp_lv_Aext) + abs(tbp_lv_B - tbp_lv_Bext)
    'shape_color_consistency',       # tbp_lv_eccentricity     * tbp_lv_color_std_mean
    'border_length_ratio',           # tbp_lv_perimeterMM      / pi * sqrt(tbp_lv_areaMM2 / pi)
    'age_size_symmetry_index',       # age_approx              * clin_size_long_diam_mm * tbp_lv_symm_2axis
    'index_age_size_symmetry',       # age_approx              * tbp_lv_areaMM2 * tbp_lv_symm_2axis
]

cat_cols = ['sex', 'anatom_site_general', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution']

norm_cols = [f'{col}_patient_norm' for col in num_cols + new_num_cols]
_patient_sum_ratio = [f'{col}_patient_sum_ratio' for col in num_cols + new_num_cols]
_patient_minmax = [f'{col}_patient_minmax' for col in num_cols + new_num_cols]
_patient_rank = [f'{col}_patient_rank' for col in num_cols + new_num_cols]
_patient_quantile_scaled = [f'{col}_patient_quantile_scaled' for col in num_cols + new_num_cols]


special_cols = ['count_per_patient']
feature_cols = num_cols + new_num_cols + cat_cols + norm_cols + special_cols + _patient_rank + _patient_minmax + _patient_sum_ratio
# _patient_minmax + _patient_quantile_scaled + _patient_sum_ratio

In [7]:
def select_features_using_corr_matrix(df, threshold=0.91):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    selected_features = df.columns.difference(to_drop)
    return selected_features.tolist()

In [8]:

def read_data(path, oof_path=None, oof_path2=None):
    df = (
        pl.read_csv(path)
        .with_columns(
            pl.col('age_approx').cast(pl.String).replace('NA', np.nan).cast(pl.Float64),
        )
        .with_columns(
            pl.col(pl.Float64).fill_nan(pl.col(pl.Float64).median()), # You may want to impute test data with train
        )
        .with_columns(
            lesion_size_ratio              = pl.col('tbp_lv_minorAxisMM') / pl.col('clin_size_long_diam_mm'),
            lesion_shape_index             = pl.col('tbp_lv_areaMM2') / (pl.col('tbp_lv_perimeterMM') ** 2),
            hue_contrast                   = (pl.col('tbp_lv_H') - pl.col('tbp_lv_Hext')).abs(),
            luminance_contrast             = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs(),
            lesion_color_difference        = (pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2).sqrt(),
            border_complexity              = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_uniformity               = pl.col('tbp_lv_color_std_mean') / (pl.col('tbp_lv_radial_color_std_max') + err),
        )
        .with_columns(
            position_distance_3d           = (pl.col('tbp_lv_x') ** 2 + pl.col('tbp_lv_y') ** 2 + pl.col('tbp_lv_z') ** 2).sqrt(),
            perimeter_to_area_ratio        = pl.col('tbp_lv_perimeterMM') / pl.col('tbp_lv_areaMM2'),
            area_to_perimeter_ratio        = pl.col('tbp_lv_areaMM2') / pl.col('tbp_lv_perimeterMM'),
            lesion_visibility_score        = pl.col('tbp_lv_deltaLBnorm') + pl.col('tbp_lv_norm_color'),
            combined_anatomical_site       = pl.col('anatom_site_general') + '_' + pl.col('tbp_lv_location'),
            symmetry_border_consistency    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border'),
            consistency_symmetry_border    = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_norm_border') / (pl.col('tbp_lv_symm_2axis') + pl.col('tbp_lv_norm_border')),
        )
        .with_columns(
            color_consistency              = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_Lext'),
            consistency_color              = pl.col('tbp_lv_stdL') * pl.col('tbp_lv_Lext') / (pl.col('tbp_lv_stdL') + pl.col('tbp_lv_Lext')),
            size_age_interaction           = pl.col('clin_size_long_diam_mm') * pl.col('age_approx'),
            hue_color_std_interaction      = pl.col('tbp_lv_H') * pl.col('tbp_lv_color_std_mean'),
            lesion_severity_index          = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_eccentricity')) / 3,
            shape_complexity_index         = pl.col('border_complexity') + pl.col('lesion_shape_index'),
            color_contrast_index           = pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL') + pl.col('tbp_lv_deltaLBnorm'),
        )
        .with_columns(
            log_lesion_area                = (pl.col('tbp_lv_areaMM2') + 1).log(),
            normalized_lesion_size         = pl.col('clin_size_long_diam_mm') / pl.col('age_approx'),
            mean_hue_difference            = (pl.col('tbp_lv_H') + pl.col('tbp_lv_Hext')) / 2,
            std_dev_contrast               = ((pl.col('tbp_lv_deltaA') ** 2 + pl.col('tbp_lv_deltaB') ** 2 + pl.col('tbp_lv_deltaL') ** 2) / 3).sqrt(),
            color_shape_composite_index    = (pl.col('tbp_lv_color_std_mean') + pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_symm_2axis')) / 3,
            lesion_orientation_3d          = pl.arctan2(pl.col('tbp_lv_y'), pl.col('tbp_lv_x')),
            overall_color_difference       = (pl.col('tbp_lv_deltaA') + pl.col('tbp_lv_deltaB') + pl.col('tbp_lv_deltaL')) / 3,
        )
        .with_columns(
            symmetry_perimeter_interaction = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_perimeterMM'),
            comprehensive_lesion_index     = (pl.col('tbp_lv_area_perim_ratio') + pl.col('tbp_lv_eccentricity') + pl.col('tbp_lv_norm_color') + pl.col('tbp_lv_symm_2axis')) / 4,
            color_variance_ratio           = pl.col('tbp_lv_color_std_mean') / pl.col('tbp_lv_stdLExt'),
            border_color_interaction       = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color'),
            border_color_interaction_2     = pl.col('tbp_lv_norm_border') * pl.col('tbp_lv_norm_color') / (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color')),
            size_color_contrast_ratio      = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_deltaLBnorm'),
            age_normalized_nevi_confidence = pl.col('tbp_lv_nevi_confidence') / pl.col('age_approx'),
            age_normalized_nevi_confidence_2 = (pl.col('clin_size_long_diam_mm')**2 + pl.col('age_approx')**2).sqrt(),
            color_asymmetry_index          = pl.col('tbp_lv_radial_color_std_max') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            volume_approximation_3d        = pl.col('tbp_lv_areaMM2') * (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2 + pl.col('tbp_lv_z')**2).sqrt(),
            color_range                    = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')).abs() + (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext')).abs() + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext')).abs(),
            shape_color_consistency        = pl.col('tbp_lv_eccentricity') * pl.col('tbp_lv_color_std_mean'),
            border_length_ratio            = pl.col('tbp_lv_perimeterMM') / (2 * np.pi * (pl.col('tbp_lv_areaMM2') / np.pi).sqrt()),
            age_size_symmetry_index        = pl.col('age_approx') * pl.col('clin_size_long_diam_mm') * pl.col('tbp_lv_symm_2axis'),
            index_age_size_symmetry        = pl.col('age_approx') * pl.col('tbp_lv_areaMM2') * pl.col('tbp_lv_symm_2axis'),
        )
        .with_columns(
            relative_luminance_variation = pl.col('tbp_lv_stdL') / pl.col('tbp_lv_L'),
            circularity_index = pl.col('tbp_lv_perimeterMM') / (2 * (np.pi * pl.col('tbp_lv_areaMM2')).sqrt()),

            asymmetry_eccentricity_product = pl.col('tbp_lv_symm_2axis') * pl.col('tbp_lv_eccentricity'),
            normalized_luminance_difference = (pl.col('tbp_lv_L') - pl.col('tbp_lv_Lext')) / (pl.col('tbp_lv_L') + pl.col('tbp_lv_Lext') + err),
            color_distance = (pl.col('tbp_lv_A') - pl.col('tbp_lv_Aext'))**2 + (pl.col('tbp_lv_B') - pl.col('tbp_lv_Bext'))**2,
            area_to_age_ratio = pl.col('tbp_lv_areaMM2') / pl.col('age_approx'),
            aspect_ratio = pl.col('clin_size_long_diam_mm') / pl.col('tbp_lv_minorAxisMM'),
            combined_severity_score = (pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_norm_color') + pl.col('lesion_severity_index')) / 3,
            nevi_severity_interaction = pl.col('tbp_lv_nevi_confidence') * pl.col('lesion_severity_index'),
            age_contrast_interaction = pl.col('age_approx') * pl.col('tbp_lv_deltaLBnorm'),
            depth_to_surface_ratio = (pl.col('tbp_lv_x')**2 + pl.col('tbp_lv_y')**2).sqrt() / pl.col('tbp_lv_z'),
            color_volume = (pl.col('tbp_lv_L') * pl.col('tbp_lv_A') * pl.col('tbp_lv_B')) ** (1/3),
            hue_chroma_product = pl.col('tbp_lv_H') * pl.col('tbp_lv_C'),
            size_deviation_from_patient_mean = pl.col('clin_size_long_diam_mm') - pl.col('clin_size_long_diam_mm').mean().over('patient_id'),
            border_complexity_score = pl.col('tbp_lv_norm_border') + pl.col('tbp_lv_symm_2axis'),
            color_to_border_ratio = pl.col('tbp_lv_norm_color') / pl.col('tbp_lv_norm_border'),
            symmetry_eccentricity_difference = pl.col('tbp_lv_symm_2axis') - pl.col('tbp_lv_eccentricity'),
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).mean().over('patient_id')) / (pl.col(col).std().over('patient_id') + err)).alias(f'{col}_patient_norm') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            (pl.col(col) / (pl.col(col).sum().over('patient_id') + err)).alias(f'{col}_patient_sum_ratio') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).min().over('patient_id')) / (pl.col(col).max().over('patient_id') - pl.col(col).min().over('patient_id') + err)).alias(f'{col}_patient_minmax') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            (pl.col(col).rank('ordinal').over('patient_id')).alias(f'{col}_patient_rank') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            ((pl.col(col) - pl.col(col).quantile(0.25).over('patient_id')) / (pl.col(col).quantile(0.75).over('patient_id') - pl.col(col).quantile(0.25).over('patient_id') + err)).alias(f'{col}_patient_quantile_scaled') for col in (num_cols + new_num_cols)
        )
        .with_columns(
            count_per_patient = pl.col('isic_id').count().over('patient_id'),
        )
        .with_columns(
            pl.col(cat_cols).cast(pl.Categorical),
        )
        .to_pandas()
    )
    
    if oof_path:
        df_effb0_oof = pd.read_csv(oof_path)
        df_effb0_oof = df_effb0_oof[['oof_predictions_effnetb0']].reset_index(drop=True)
        df = df.reset_index(drop=True)
        df['oof_predictions_effnetb0'] = df_effb0_oof['oof_predictions_effnetb0']
        feature_cols.append('oof_predictions_effnetb0')

    
    if oof_path2:
        df_resnet18_oof = pd.read_csv(oof_path2)
        df_resnet18_oof = df_resnet18_oof[['oof_predictions_eva02']].reset_index(drop=True)
        df = df.reset_index(drop=True)
        df['oof_predictions_eva02'] = df_resnet18_oof['oof_predictions_eva02']
        feature_cols.append('oof_predictions_eva02')
    
    return df

In [9]:
# # Open the pickle file in binary read mode
# with open('/kaggle/input/shap-features-90/lgb_90_selected_features.pkl', 'rb') as file:
#     # Load the data from the file
#     lgb_shap_features = pickle.load(file)
    
# # Open the pickle file in binary read mode
# with open('/kaggle/input/shap-features-90/cb_90_selected_features.pkl', 'rb') as file:
#     # Load the data from the file
#     cb_shap_features = pickle.load(file)
    
# # Open the pickle file in binary read mode
# with open('/kaggle/input/shap-features-90/xgb_90_selected_features.pkl', 'rb') as file:
#     # Load the data from the file
#     xgb_shap_features = pickle.load(file)
    
# lgb_shap_features += ['oof_predictions_effnetb0']
# xgb_shap_features += ['oof_predictions_effnetb0']
# cb_shap_features += ['oof_predictions_effnetb0']

In [10]:
def preprocess(df_train, df_test):
    global cat_cols
    
    encoder = OneHotEncoder(sparse_output=False, dtype=np.int32, handle_unknown='ignore')
    encoder.fit(df_train[cat_cols])
    
    new_cat_cols = [f'onehot_{i}' for i in range(len(encoder.get_feature_names_out()))]

    df_train[new_cat_cols] = encoder.transform(df_train[cat_cols])
    df_train[new_cat_cols] = df_train[new_cat_cols].astype('category')

    df_test[new_cat_cols] = encoder.transform(df_test[cat_cols])
    df_test[new_cat_cols] = df_test[new_cat_cols].astype('category')

    for col in cat_cols:
        feature_cols.remove(col)

    feature_cols.extend(new_cat_cols)
#     lgb_shap_features.extend(new_cat_cols)
#     xgb_shap_features.extend(new_cat_cols)
#     cb_shap_features.extend(new_cat_cols)
    selected_features.extend(new_cat_cols)
    cat_cols = new_cat_cols
    
    return df_train, df_test

In [11]:
def custom_metric(estimator, X, y_true):
    y_hat = estimator.predict_proba(X)[:, 1]
    min_tpr = 0.80
    max_fpr = abs(1 - min_tpr)
    
    v_gt = abs(y_true - 1)
    v_pred = np.array([1.0 - x for x in y_hat])
    
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return partial_auc

In [12]:
# Usage
# , oof_path2='/kaggle/input/384-5-fold-resnest-oof-predictions/oof_predictions_resnest101.csv'
df_train = read_data(train_path, oof_path='/kaggle/input/384x384-1-50-ratio-5fold-effnetb0-oof-predictions/oof_predictions.csv')
df_test = read_data(test_path)

# hog_df = pd.read_csv('/kaggle/input/hog-featuresv2/hog_features (3).csv')
# hog_df = hog_df.iloc[:, :25]
# hog_df['image_id'] = hog_df['image_id'].str.replace('.jpg', '')

# df_train = pd.merge(df_train, hog_df, how='outer', left_on='isic_id', right_on='image_id')
# df_train = df_train.drop(columns=['image_id'])

# hog_num_cols = hog_df.select_dtypes(include='number').columns.tolist()

def ugly_duckling_processing(df, num_cols):
    ud_columns = num_cols.copy()
    ud_num_cols = []
    
    #if false - only do location-based ugly ducklings
    include_patient_wide_ud = False  
    
    counter = 0
    
    def calc_ugly_duckling_scores(group, grouping):
        nonlocal counter
        counter += 1
        if counter % 10 == 0: print(".", end="", flush=True)
        z_scores = group[ud_columns].apply(lambda x: zscore(x, nan_policy='omit'))
        ud_scores = np.abs(z_scores)
        prefix = 'ud_' if grouping == 'patient' else 'ud_loc_'
        ud_scores.columns = [f'{prefix}{col}' for col in ud_columns]
        return ud_scores

    print("Analyzing ducklings", end="", flush=True)
    ud_location_col = 'tbp_lv_location'
    ud_scores_loc = df.groupby(['patient_id', ud_location_col])[ud_columns + ['patient_id', ud_location_col]].apply(
        lambda x: calc_ugly_duckling_scores(x, 'location')
    ).reset_index(level=[0, 1], drop=True)
    
    print("\nConcat ducklings")
    df = pd.concat([df, ud_scores_loc], axis=1)
    
    if include_patient_wide_ud:
        print("Analyzing ducklings (part 2)", end="", flush=True)
        ud_scores_patient = df.groupby('patient_id')[ud_columns + ['patient_id']].apply(
            lambda x: calc_ugly_duckling_scores(x, 'patient')
        ).reset_index(level=0, drop=True)
        df = pd.concat([df, ud_scores_patient], axis=1)
        print()  # New line after progress indicator

    print("Extending ducklings")
    ud_num_cols.extend([f'ud_loc_{col}' for col in ud_columns])
    if include_patient_wide_ud:
        ud_num_cols.extend([f'ud_{col}' for col in ud_columns])

    print("Enhancing ugly duckling features", end="", flush=True)
    
    # 1. Percentile-based ugly duckling scores
    def calc_percentile_ud_scores(group):
        nonlocal counter
        counter += 1
        if counter % 10 == 0: print(".", end="", flush=True)
        percentiles = group[ud_columns].rank(pct=True)
        return percentiles.add_prefix('ud_percentile_')
    
    counter = 0  # Reset counter for percentile calculation
    ud_percentiles = df.groupby('patient_id')[ud_columns].apply(calc_percentile_ud_scores).reset_index(level=0, drop=True)
    df = pd.concat([df, ud_percentiles], axis=1)
    ud_num_cols.extend([f'ud_percentile_{col}' for col in ud_columns])
    print()  # New line after progress indicator

    # 2. Ugly duckling count features
    threshold = 2.0  # You can adjust this threshold
    if include_patient_wide_ud:
        ud_count = (df[[f'ud_{col}' for col in ud_columns]].abs() > threshold).sum(axis=1)
        df['ud_count_patient'] = ud_count
        ud_num_cols.append('ud_count_patient')
    
    ud_count_loc = (df[[f'ud_loc_{col}' for col in ud_columns]].abs() > threshold).sum(axis=1)
    df['ud_count_location'] = ud_count_loc
    ud_num_cols.append('ud_count_location')

    # 3. Ugly duckling severity features
    if include_patient_wide_ud:
        df['ud_max_severity_patient'] = df[[f'ud_{col}' for col in ud_columns]].abs().max(axis=1)
        ud_num_cols.append('ud_max_severity_patient')
    df['ud_max_severity_location'] = df[[f'ud_loc_{col}' for col in ud_columns]].abs().max(axis=1)
    ud_num_cols.append('ud_max_severity_location')

    # 4. Ugly duckling consistency features
    if include_patient_wide_ud:
        df['ud_consistency_patient'] = df[[f'ud_{col}' for col in ud_columns]].abs().std(axis=1)
        ud_num_cols.append('ud_consistency_patient')
    df['ud_consistency_location'] = df[[f'ud_loc_{col}' for col in ud_columns]].abs().std(axis=1)
    ud_num_cols.append('ud_consistency_location')

    return df, ud_num_cols

if do_ud:
    df_train, ud_num_cols = ugly_duckling_processing(df_train.copy(), num_cols+new_num_cols)
    df_test, _ = ugly_duckling_processing(df_test.copy(), num_cols+new_num_cols)

Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.


Analyzing ducklings

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Concat ducklings
Extending ducklings
Enhancing ugly duckling features


In [13]:

    
num_cols = num_cols + new_num_cols + norm_cols + special_cols + _patient_rank + ud_num_cols + _patient_minmax + _patient_sum_ratio
# _patient_quantile_scaled + _patient_minmax + lbp_num_cols + _patient_sum_ratio

selected_features = select_features_using_corr_matrix(df_train[num_cols])
print(len(selected_features))
    
df_train, df_test = preprocess(df_train, df_test)
print(len(selected_features))

191
238


In [14]:
print(len(selected_features))
selected_features += ['oof_predictions_effnetb0']
# lgb_shap_features += ['oof_predictions_effnetb0']
# xgb_shap_features += ['oof_predictions_effnetb0']
# cb_shap_features += ['oof_predictions_effnetb0']
# print(len(lgb_shap_features))

print(len(selected_features))

238
239


### Data Read & Feature Engineering

In [15]:
X = df_train[selected_features]
y = df_train[target_col]
groups = df_train[group_col]
# cv = StratifiedGroupKFold(5, shuffle=True, random_state=seed)


N_SPLITS = 5
gkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

df_train["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx
    

# val_score = cross_val_score(
#     estimator=estimator, 
#     X=X, y=y, 
#     cv=cv, 
#     groups=groups,
#     scoring=custom_metric,
# )

# np.mean(val_score), val_score

In [16]:
df_train['fold'].tail()

401054    4
401055    3
401056    1
401057    3
401058    0
Name: fold, dtype: int64

In [17]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [18]:

lgb_params = {
        'objective':        'binary',
        'verbosity':        -1,
        'n_iter':           200,
        'boosting_type':    'gbdt',
        'random_state':     seed,
        'learning_rate': 0.03104273262811841,
        'num_leaves': 111,
        'min_child_samples': 100,
        'bagging_fraction': 0.9101945804571369,
        'feature_fraction': 0.34103959543345147,
        'feature_fraction_bynode ': 0.5450587760167019,
        'bagging_freq': 1,
        'lambda_l1': 3.211848972246674e-07, 
        'lambda_l2': 3.720742147032093e-07,
        'max_depth': 4,
        'scale_pos_weight': 2.785263418574575,
    # 0.1823
}


cb_params = {
    'loss_function':     'Logloss',
    'iterations':        200,
    'verbose':           False,
    'random_state':      seed,
    'cat_features':      cat_cols,
    'depth':             5,
    'learning_rate':     0.0721506164096434,
    'l2_leaf_reg':       7.475812134744556,
    'min_data_in_leaf':  59,
    'scale_pos_weight':  4.254188566545996,
    'subsample':         0.6016652239201599
}

xgb_params = {
    'enable_categorical': True,
    'tree_method':        'hist',
    'random_state':       seed,
    'n_estimators':       206,
    'learning_rate':      0.042310779751030335, 
    'lambda':             0.0002847007250281572, 
    'alpha':              4.478266669867162e-08, 
    'max_depth':          6, 
    'subsample':          0.811997027872852, 
    'min_child_weight':   6,
#     'colsample_bytree':   0.8437772277074493, 
#     'colsample_bylevel':  0.5476090898823716, 
#     'colsample_bynode':   0.9928601203635129, 
    'scale_pos_weight':   3.884899202547225,
}




In [19]:
from tqdm import tqdm  # For progress bar

if SHAP_LGB:

    # Initialize variables
    lgb_scores = []
    lgb_models = []
    oof_df = pd.DataFrame()
    lgbm_shap_values_list = []

    # Train model and calculate SHAP values for each fold
    for fold in range(N_SPLITS):
        _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
        _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

        lgb_model = Pipeline([
            ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
            ('classifier', lgb.LGBMClassifier(**lgb_params)),
        ])
        lgb_model.fit(_df_train[selected_features], _df_train["target"])

        preds = lgb_model.predict_proba(_df_valid[selected_features])[:, 1]
        score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
        print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
        lgb_models.append(lgb_model)

        # Collect out-of-fold predictions
        oof_single = _df_valid[["isic_id", "target"]].copy()
        oof_single["pred"] = preds
        oof_df = pd.concat([oof_df, oof_single])

        # SHAP values calculation
        lgb_model_step = lgb_model.named_steps['classifier']  # Access LightGBM model from the pipeline
        explainer = shap.TreeExplainer(lgb_model_step)
        lgbm_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
        lgbm_shap_values_list.append(lgbm_shap_values)

    # Aggregate SHAP values across all folds
    shap_values_array = np.vstack(lgbm_shap_values_list)
    mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)

    # Create a DataFrame to rank features by SHAP importance
    shap_importance_df = pd.DataFrame({
        'feature': selected_features,
        'mean_abs_shap_value': mean_abs_shap_values
    }).sort_values(by='mean_abs_shap_value', ascending=False)

    # Define the range of N_TOP_FEATURES to evaluate
    feature_range = range(3, len(selected_features), 3)  # Evaluate every 10 features up to the number of selected features
    feature_perf_scores = {}

    # Evaluate performance for different numbers of top features
    for N_TOP_FEATURES in tqdm(feature_range, desc="Evaluating N_TOP_FEATURES"):
        selected_features_subset = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()

        # Cross-validate using only top N_TOP_FEATURES
        fold_scores = []
        for fold in range(N_SPLITS):
            _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
            _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

            # Model pipeline with selected features only
            lgb_model = Pipeline([
                ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
                ('classifier', lgb.LGBMClassifier(**lgb_params)),
            ])
            lgb_model.fit(_df_train[selected_features_subset], _df_train["target"])

            preds = lgb_model.predict_proba(_df_valid[selected_features_subset])[:, 1]
            score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
            
            
            oof_single = _df_valid[["isic_id", "target"]].copy()
            oof_single["pred"] = preds
            oof_df = pd.concat([oof_df, oof_single])

        # Store the average performance score for the current N_TOP_FEATURES
        lgbm_score_oof = comp_score(oof_df["target"], oof_df["pred"], "")
        feature_perf_scores[N_TOP_FEATURES] = lgbm_score_oof
        print(f"Top {N_TOP_FEATURES} features: OOF LGBM Partial AUC Score: {lgbm_score_oof:.5f}")

    # Find the optimal N_TOP_FEATURES
    optimal_features = max(feature_perf_scores, key=feature_perf_scores.get)
    print(f"Optimal number of top features: {optimal_features} with Partial AUC Score: {feature_perf_scores[optimal_features]:.5f}")

    # Select the optimal features
    lgbm_selected_features = shap_importance_df['feature'].head(optimal_features).tolist()
    print(f"Selected top {optimal_features} features: {lgbm_selected_features}")

    # Save the optimal SHAP values list and selected features to files
    with open('lgbm_shap_values_list.pkl', 'wb') as f:
        pickle.dump(lgbm_shap_values_list, f)
    with open(f'lgbm_selected_features.pkl_{optimal_features}', 'wb') as f:
        pickle.dump(lgbm_selected_features, f)

In [20]:
if SHAP_CB:

    cb_scores = []
    cb_models = []
    oof_df = pd.DataFrame()
    cb_shap_values_list = []

    for fold in range(N_SPLITS):
        _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
        _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

        cb_model = Pipeline([
              ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
              ('classifier', cb.CatBoostClassifier(**cb_params)),
          ])

        cb_model.fit(_df_train[selected_features], _df_train["target"])
        preds = cb_model.predict_proba(_df_valid[selected_features])[:, 1]
        score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")

        print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
        cb_models.append(cb_model)
        oof_single = _df_valid[["isic_id", "target"]].copy()
        oof_single["pred"] = preds
        oof_df = pd.concat([oof_df, oof_single])

          # SHAP values calculation
        cb_model_step = cb_model.named_steps['classifier']
        explainer = shap.TreeExplainer(cb_model_step)
        cb_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
        cb_shap_values_list.append(cb_shap_values)

    shap_values_array = np.vstack(cb_shap_values_list)
    mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)

    shap_importance_df = pd.DataFrame({
          'feature': selected_features,
          'mean_abs_shap_value': mean_abs_shap_values
    }).sort_values(by='mean_abs_shap_value', ascending=False)

    feature_range = range(3, len(selected_features), 3)
    feature_perf_scores = {}

    for N_TOP_FEATURES in tqdm(feature_range, desc="Evaluating N_TOP_FEATURES"):
        selected_features_subset = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()

        fold_scores = []
        for fold in range(N_SPLITS):
            _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
            _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

            cb_model = Pipeline([
                  ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
                  ('classifier', cb.CatBoostClassifier(**cb_params)),
            ])
            cb_model.fit(_df_train[selected_features_subset], _df_train["target"])
            preds = cb_model.predict_proba(_df_valid[selected_features_subset])[:, 1]
            score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
            
                        
            oof_single = _df_valid[["isic_id", "target"]].copy()
            oof_single["pred"] = preds
            oof_df = pd.concat([oof_df, oof_single])

        cb_score_oof = comp_score(oof_df["target"], oof_df["pred"], "")
        feature_perf_scores[N_TOP_FEATURES] = cb_score_oof
        print(f"Top {N_TOP_FEATURES} features: Average Partial AUC Score: {cb_score_oof:.5f}")

    optimal_features = max(feature_perf_scores, key=feature_perf_scores.get)
    print(f"Optimal number of top features: {optimal_features} with Partial AUC Score: {feature_perf_scores[optimal_features]:.5f}")

    cb_selected_features = shap_importance_df['feature'].head(optimal_features).tolist()
    print(f"Selected top {optimal_features} features: {cb_selected_features}")

    with open('cb_shap_values_list.pkl', 'wb') as f:
        pickle.dump(cb_shap_values_list, f)

    with open(f'cb_selected_features.pkl_{optimal_features}', 'wb') as f:
        pickle.dump(cb_selected_features, f)


In [21]:
if SHAP_XGB:

    # Initialize variables
    xgb_scores = []
    xgb_models = []
    oof_df = pd.DataFrame()
    xgb_shap_values_list = []

    for fold in range(N_SPLITS):
        _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
        _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

        # Train model

        xgb_model = Pipeline([
        #     ('feature_selection', xgb_transformer),
            ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
            ('classifier', xgb.XGBClassifier(**xgb_params)),
        ])
        xgb_model.fit(_df_train[selected_features], _df_train["target"])

        preds = xgb_model.predict_proba(_df_valid[selected_features])[:, 1]
        score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
        print(f"fold: {fold} - Partial AUC Score: {score:.5f}")

        xgb_models.append(xgb_model)

        # Collect out-of-fold predictions
        oof_single = _df_valid[["isic_id", "target"]].copy()
        oof_single["pred"] = preds
        oof_df = pd.concat([oof_df, oof_single])

        # SHAP values calculation
        xgb_model_step = xgb_model.named_steps['classifier']  # Access LightGBM model from the pipeline
        explainer = shap.TreeExplainer(xgb_model_step)
        xgb_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
        xgb_shap_values_list.append(xgb_shap_values)

    shap_values_array = np.vstack(xgb_shap_values_list)
    mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)

    shap_importance_df = pd.DataFrame({
        'feature': selected_features,
        'mean_abs_shap_value': mean_abs_shap_values
    }).sort_values(by='mean_abs_shap_value', ascending=False)

    feature_range = range(3, len(selected_features), 3)  # Evaluate every 10 features up to the number of selected features
    feature_perf_scores = {}

    for N_TOP_FEATURES in tqdm(feature_range, desc="Evaluating N_TOP_FEATURES"):
        selected_features_subset = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()

        fold_scores = []

        for fold in range(N_SPLITS):
            _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
            _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

            xgb_model = Pipeline([
                    ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
                    ('classifier', xgb.XGBClassifier(**xgb_params)),
            ])
                
            xgb_model.fit(_df_train[selected_features_subset], _df_train["target"])

            preds = xgb_model.predict_proba(_df_valid[selected_features_subset])[:, 1]
            score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
            
            oof_single = _df_valid[["isic_id", "target"]].copy()
            oof_single["pred"] = preds
            oof_df = pd.concat([oof_df, oof_single])


        xgb_score_oof = comp_score(oof_df["target"], oof_df["pred"], "")
        feature_perf_scores[N_TOP_FEATURES] = xgb_score_oof
        print(f"Top {N_TOP_FEATURES} features: Average Partial AUC Score: {xgb_score_oof:.5f}")

    optimal_features = max(feature_perf_scores, key=feature_perf_scores.get)
    print(f"Optimal number of top features: {optimal_features} with Partial AUC Score: {feature_perf_scores[optimal_features]:.5f}")

    xgb_selected_features = shap_importance_df['feature'].head(optimal_features).tolist()
    print(f"Selected top {optimal_features} features: {xgb_selected_features}")

    with open('xgb_shap_values_list.pkl', 'wb') as f:
        pickle.dump(xgb_shap_values_list, f)

    with open(f'xgb_selected_features.pkl_{optimal_features}', 'wb') as f:
        pickle.dump(xgb_selected_features, f)
        
                                                                

fold: 0 - Partial AUC Score: 0.18033
fold: 1 - Partial AUC Score: 0.17732
fold: 2 - Partial AUC Score: 0.18803
fold: 3 - Partial AUC Score: 0.17892
fold: 4 - Partial AUC Score: 0.18830


Evaluating N_TOP_FEATURES:   1%|▏         | 1/79 [00:26<34:15, 26.35s/it]

Top 3 features: Average Partial AUC Score: 0.16778


Evaluating N_TOP_FEATURES:   3%|▎         | 2/79 [00:50<32:05, 25.00s/it]

Top 6 features: Average Partial AUC Score: 0.16695


...

Top 231 features: Average Partial AUC Score: 0.18057


Evaluating N_TOP_FEATURES:  99%|█████████▊| 78/79 [1:57:23<02:35, 155.63s/it]

Top 234 features: Average Partial AUC Score: 0.18058


Evaluating N_TOP_FEATURES: 100%|██████████| 79/79 [1:59:59<00:00, 91.14s/it] 

Top 237 features: Average Partial AUC Score: 0.18059
Optimal number of top features: 237 with Partial AUC Score: 0.18059
Selected top 237 features: ['oof_predictions_effnetb0', 'age_approx_patient_rank', 'tbp_lv_H_patient_rank', 'onehot_45', 'tbp_lv_deltaB_patient_rank', 'age_normalized_nevi_confidence_2_patient_norm', 'ud_percentile_hue_contrast', 'ud_max_severity_location', 'ud_loc_tbp_lv_y', 'tbp_lv_y_patient_norm', 'ud_loc_clin_size_long_diam_mm', 'clin_size_long_diam_mm', 'tbp_lv_H_patient_norm', 'ud_loc_border_color_interaction_2', 'ud_loc_tbp_lv_z', 'tbp_lv_deltaL_patient_norm', 'ud_percentile_age_approx', 'tbp_lv_deltaL_patient_rank', 'ud_consistency_location', 'color_uniformity_patient_norm', 'tbp_lv_y_patient_rank', 'tbp_lv_Aext', 'ud_loc_hue_contrast', 'tbp_lv_nevi_confidence', 'ud_loc_mean_hue_difference', 'age_approx', 'ud_loc_tbp_lv_eccentricity', 'lesion_orientation_3d_patient_minmax', 'tbp_lv_Hext_patient_minmax', 'tbp_lv_H_patient_minmax', 'tbp_lv_stdLExt_patient_minma

### SHAP Tuned Models

In [22]:
# if SHAP:
    
#     lgb_params = {
#         'objective':        'binary',
#         'verbosity':        -1,
#         'n_iter':           200,
#         'boosting_type':    'gbdt',
#         'random_state':     seed,
#         'learning_rate':    0.03105060670990981,
#         'num_leaves':       23,
#         'min_child_samples':80,
#         'bagging_fraction': 0.7649096623770557,
#         'bagging_freq':     2,
#         'lambda_l1':        0.1092109244738963,
#         'lambda_l2':        0.0001329432266572826,
#         'max_depth':        4,
#         'scale_pos_weight': 3.8792378580090796
#     }

#     lgb_scores = []
#     lgb_models = []
#     oof_df = pd.DataFrame()
#     lgbm_shap_values_list = []

#     for fold in range(N_SPLITS):
#         _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
#         _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

#         lgb_model = Pipeline([
#         #     ('feature_selection', lgb_transformer),
#             ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
#             ('classifier', lgb.LGBMClassifier(**lgb_params)),
#         ])
#         lgb_model.fit(_df_train[selected_features], _df_train["target"])

#         preds = lgb_model.predict_proba(_df_valid[selected_features])[:, 1]
#         score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
#         print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
#         lgb_models.append(lgb_model)
#         oof_single = _df_valid[["isic_id", "target"]].copy()
#         oof_single["pred"] = preds
#         oof_df = pd.concat([oof_df, oof_single])

#         # SHAP values calculation for feature selection
#         lgb_model_step = lgb_model.named_steps['classifier']  # Access the LightGBM model from the pipeline
#         explainer = shap.TreeExplainer(lgb_model_step)
#         lgbm_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
#         lgbm_shap_values_list.append(lgbm_shap_values)
    
#     # Aggregate SHAP values across all folds
#     shap_values_array = np.vstack(lgbm_shap_values_list)
#     mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)
    
#     # Create a DataFrame to rank features
#     shap_importance_df = pd.DataFrame({
#         'feature': selected_features,
#         'mean_abs_shap_value': mean_abs_shap_values
#     }).sort_values(by='mean_abs_shap_value', ascending=False)
    
#     # Select top N features based on SHAP importance
#     N_TOP_FEATURES = 75  # Define how many top features you want to select
#     lgbm_selected_features = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()
    
#     print(f"Selected top {N_TOP_FEATURES} features: {lgbm_selected_features}")
    
#     # Save SHAP values list and selected features to files
#     with open('lgbm_shap_values_list.pkl', 'wb') as f:
#         pickle.dump(lgbm_shap_values_list, f)
#     with open(f'lgbm_selected_features.pkl_{N_TOP_FEATURES}', 'wb') as f:
#         pickle.dump(lgbm_selected_features, f)

In [23]:
# if SHAP:
    
#     cb_params = {
#         'loss_function':     'Logloss',
#         'iterations':        200,
#         'verbose':           False,
#         'random_state':      seed,
# #         'cat_features':      cat_cols,
#         'depth':             5,
#         'learning_rate':     0.0721506164096434,
#         'l2_leaf_reg':       7.475812134744556,
#         'min_data_in_leaf':  59,
#         'scale_pos_weight':  4.254188566545996,
#         'subsample':         0.6016652239201599
#     }
    
#     cb_scores = []
#     cb_models = []
#     oof_df = pd.DataFrame()
#     cb_shap_values_list = []

#     for fold in range(N_SPLITS):
#         _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
#         _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

#         # Train model
        
#         cb_model = Pipeline([
#             ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
#             ('classifier', cb.CatBoostClassifier(**cb_params)),
#         ])
#         cb_model.fit(_df_train[selected_features], _df_train["target"])

#         preds = cb_model.predict_proba(_df_valid[selected_features])[:, 1]
#         score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
#         print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
#         cb_models.append(cb_model)
#         oof_single = _df_valid[["isic_id", "target"]].copy()
#         oof_single["pred"] = preds
#         oof_df = pd.concat([oof_df, oof_single])

#         # SHAP values calculation for feature selection
#         cb_model_step = cb_model.named_steps['classifier']  # Access the LightGBM model from the pipeline
#         explainer = shap.TreeExplainer(cb_model_step)
#         cb_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
#         cb_shap_values_list.append(cb_shap_values)
    
#     # Aggregate SHAP values across all folds
#     shap_values_array = np.vstack(cb_shap_values_list)
#     mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)
    
#     # Create a DataFrame to rank features
#     shap_importance_df = pd.DataFrame({
#         'feature': selected_features,
#         'mean_abs_shap_value': mean_abs_shap_values
#     }).sort_values(by='mean_abs_shap_value', ascending=False)
    
#     # Select top N features based on SHAP importance
#     N_TOP_FEATURES = 75  # Define how many top features you want to select
#     cb_selected_features = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()
    
#     print(f"Selected top {N_TOP_FEATURES} features: {cb_selected_features}")
    
#     # Save SHAP values list and selected features to files
#     with open('cb_shap_values_list.pkl', 'wb') as f:
#         pickle.dump(cb_shap_values_list, f)
#     with open(f'cb_selected_features.pkl_{N_TOP_FEATURES}', 'wb') as f:
#         pickle.dump(cb_selected_features, f)

In [24]:
# if SHAP:
    
#     xgb_params = {
#         'enable_categorical': True,
#         'tree_method':        'hist',
#         'random_state':       seed,
#         'learning_rate':      0.042310779751030335, 
#         'lambda':             0.0002847007250281572, 
#         'alpha':              4.478266669867162e-08, 
#         'max_depth':          6, 
#         'subsample':          0.811997027872852, 
#         'min_child_weight':   6,
#     #     'colsample_bytree':   0.8437772277074493, 
#     #     'colsample_bylevel':  0.5476090898823716, 
#     #     'colsample_bynode':   0.9928601203635129, 
#         'scale_pos_weight':   3.884899202547225,
#     }


#     xgb_scores = []
#     xgb_models = []
#     oof_df = pd.DataFrame()
#     xgb_shap_values_list = []

#     for fold in range(N_SPLITS):
#         _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
#         _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)

#         # Train model
        
#         xgb_model = Pipeline([
#         #     ('feature_selection', xgb_transformer),
#             ('sampler', RandomUnderSampler(sampling_strategy=sampling_ratio, random_state=seed)),
#             ('classifier', xgb.XGBClassifier(**xgb_params)),
#         ])
#         xgb_model.fit(_df_train[selected_features], _df_train["target"])

#         preds = xgb_model.predict_proba(_df_valid[selected_features])[:, 1]
#         score = comp_score(_df_valid[["target"]], pd.DataFrame(preds, columns=["prediction"]), "")
#         print(f"fold: {fold} - Partial AUC Score: {score:.5f}")
#         xgb_models.append(xgb_model)
#         oof_single = _df_valid[["isic_id", "target"]].copy()
#         oof_single["pred"] = preds
#         oof_df = pd.concat([oof_df, oof_single])

#         # SHAP values calculation for feature selection
#         xgb_model_step = xgb_model.named_steps['classifier']  # Access the LightGBM model from the pipeline
#         explainer = shap.TreeExplainer(xgb_model_step)
#         xgb_shap_values = explainer.shap_values(_df_valid[selected_features])[1]
#         xgb_shap_values_list.append(xgb_shap_values)
    
#     # Aggregate SHAP values across all folds
#     shap_values_array = np.vstack(xgb_shap_values_list)
#     mean_abs_shap_values = np.mean(np.abs(shap_values_array), axis=0)
    
#     # Create a DataFrame to rank features
#     shap_importance_df = pd.DataFrame({
#         'feature': selected_features,
#         'mean_abs_shap_value': mean_abs_shap_values
#     }).sort_values(by='mean_abs_shap_value', ascending=False)
    
#     # Select top N features based on SHAP importance
#     N_TOP_FEATURES = 75  # Define how many top features you want to select
#     xgb_selected_features = shap_importance_df['feature'].head(N_TOP_FEATURES).tolist()
    
#     print(f"Selected top {N_TOP_FEATURES} features: {xgb_selected_features}")
    
#     # Save SHAP values list and selected features to files
#     with open('xgb_shap_values_list.pkl', 'wb') as f:
#         pickle.dump(xgb_shap_values_list, f)
#     with open(f'xgb_selected_features.pkl_{N_TOP_FEATURES}', 'wb') as f:
#         pickle.dump(xgb_selected_features, f)